# Understanding molecular bases of drug response and drug synergy
Understanding synergistic effects of drugs is key to develop effective intervention strategies targeting diseases (such as AD or T2D or both) and provides unprecedented opportunities to repurpose existing drugs. The AstraZeneca-Sanger Drug Combination Prediction DREAM Challenge provides a rich data source aiming to understand the synergistic drug behavior based on pretreatment data and spans cell viability data over 118 drugs and 85 cancer cell lines (primarily colon, lung, and breast). In collaborating with Dr. Baldo Oliva's group at GRIB, UPF-IMIM, we have been working on identifying the effects of confounding factors in the data set such as dosage and genetic background of the cell lines and developing algorithms that can predict the individual and synergistic effects of drugs. 

The challenge has two subtasks: predicting drug synergy *(i)* using mono synergy data *(ii)* without using mono synergy data. The participants are free to use any other data source (such as cell line gene expression, gene mutation, drug target information provided  in the challenge or external data sets) and submit their predictions in 3 rounds. 

For the first round of the challenge, we have build machine learning models to predict the synergy of drugs for both of these tasks and choice the best performing models to submit predictions. Among various machine learning models, we found a combination of bootstrapped and ensemble tree-based predictors achieved best performance on the training data set for. 

To improve the prediction performance we have incorporated mutation data (of drug targets in a given cell line) and interactome based contribution of the drug combination compared to the effect of drugs separately. To assess interactome based contribution of a drug or combination (characterized by a set of targets), we have used GUILD, a network-based functional prioritization tool. 

Interestingly, using GUILD, only the predictions for subtask *(ii)* improved but not for subtask *(i)*. We suspect this is due to the mono therapy response data describing the synergy best and addition of new features (such as the ones based on expression, mutation, interactome) potentially causing the predictor to overfit to the training data set.